In [1]:
import json

In [4]:
class WordPieceTokenizer:
    def __init__(self,corpus_file_path,path_for_vocab_txt_out,sentences_id_in_json,token_id_out_json,iterations,max_vocab_size):
        self.corpus_file_path=corpus_file_path
        self.iterations=iterations
        self.vocabulary=None
        self.path_for_vocab_txt_out=path_for_vocab_txt_out
        self.max_vocab_size=max_vocab_size
        self.sentences_id_in_json=sentences_id_in_json
        self.token_id_out_json=token_id_out_json
        with open(corpus_file_path, 'r') as file:
            self.corpus=file.read()
        self.corpus=self.corpus
    def extract_words(self):
        words_freq={}
        word=""
        for character in self.corpus:
            if 'a'<=character and character<='z':
                word+=character
            else:
                if word  != "":
                    if word not in words_freq:
                        words_freq[word]=1
                        word=""
                    else:
                        words_freq[word]+=1
                        word=""
        if word != "":
            if word not in words_freq:
                words_freq[word]=1
            else:
                words_freq[word]+= 1


        print(words_freq)

        word_splits={}
        for i in words_freq.keys():
            word_temp=[]
            o=True
            for k in i:
                if o:
                    word_temp.append(k)
                    o=False
                else:
                    word_temp.append("##"+k)
            word_splits[i]=word_temp


        print(word_splits,words_freq)
        return word_splits,words_freq
    def init_vocab(self, word_split):
        vocab_init=[]
        for i,j in word_split.items():
            for k in j:
                if k not in vocab_init:
                    vocab_init.append(k)
        return vocab_init
    def compute_pair_score(self, words_freq, word_splits):
        pair_freq={}
        letter_freq={}
        for word,freq in words_freq.items():
            split_of_word=word_splits[word]
            if(len(split_of_word)==1):
                if split_of_word[0] not in letter_freq:
                    letter_freq[split_of_word[0]]=freq
                else:
                    letter_freq[split_of_word[0]]+=freq
            else:
                for i in range(len(split_of_word)-1):
                    pair_of_split=(split_of_word[i],split_of_word[i+1])
                    if pair_of_split not in pair_freq:
                        pair_freq[pair_of_split]=freq
                    else:
                         pair_freq[pair_of_split]+=freq
                    # pair_freq[pair_of_split]+=freq
                    if split_of_word[i] not in letter_freq:
                        letter_freq[split_of_word[i]]=freq
                    else:
                        letter_freq[split_of_word[i]]+=freq
                    
                    # letter_freq[split_of_word[i]]+=freq
                if split_of_word[-1] not in letter_freq:
                    letter_freq[split_of_word[-1]]=freq
                else:
                    letter_freq[split_of_word[-1]]+=freq
                # letter_freq[split_of_word[-1]]+=freq
        scores={}
        for pair_word,freq in pair_freq.items():
            scores[pair_word]=freq/(letter_freq[pair_word[0]]*letter_freq[pair_word[1]])
        return scores
    def merge_pair(self,a,b, splits):
        for word_temp, split_temp in splits.items():
            split_temp_loop=split_temp
            if len(split_temp_loop)>1:
                i=0
                while i<len(split_temp_loop)-1:
                    if split_temp_loop[i]==a and split_temp_loop[i+1]==b:
                        merge=""
                        if b[0:2]=="##":
                            merge=a+b[2:]
                        else:
                            merge=a+b
                        split_temp_loop=split_temp_loop[:i]+[merge]+split_temp_loop[i+2:]
                    else:
                        i+=1
                splits[word_temp]=split_temp_loop
        return splits
    def construct_vocabulary(self):
        word_splits,words_freq=self.extract_words()
        vocab_temp=self.init_vocab(word_splits)
        for i in range(self.iterations):
            score_temp=self.compute_pair_score(words_freq,word_splits)
            best_pair=None
            max_score=None
            for pair_temp,prob_temp in score_temp.items():
                if max_score is None or max_score<prob_temp:
                    best_pair=pair_temp
                    max_score=prob_temp
            word_splits=self.merge_pair(best_pair[0],best_pair[1],word_splits)
            token_to_insert=""
            if(best_pair[1][0:2]=="##"):
                token_to_insert=best_pair[0]+best_pair[1][2:]
            else:
                token_to_insert=best_pair[0]+best_pair[1]
            if token_to_insert not in vocab_temp:
                vocab_temp.append(token_to_insert)
        print("__________")
        self.vocabulary=vocab_temp
        self.write_vocab_to_file()
        return vocab_temp
    

    def tokenize_word(self,word_to_tokenize):
        tokens_res=[]
        start_ind=0
        while start_ind<len(word_to_tokenize):
            end_ind=len(word_to_tokenize)

            found_token=False

            while start_ind<end_ind:
                subword_to_find=word_to_tokenize[start_ind:end_ind]
                if start_ind>0:
                    subword_to_find="##"+subword_to_find

                if subword_to_find in self.vocabulary:
                    tokens_res.append(subword_to_find)
                    start_ind=end_ind
                    found_token=True
                    break
                end_ind-=1
            if not found_token:
                tokens_res.append("[UNK]")
                start_ind+=1
        return tokens_res
    def tokenize_sentence(self, text_to_tokenize):
        sentence_to_tokenize=[]
        word=""
        for character in text_to_tokenize:
            if 'a'<=character and character<='z':
                word+=character
            else:
                if word  != "":
                    sentence_to_tokenize.append(word)
                    word=""
                    
        if word != "":
            sentence_to_tokenize.append(word)
        res_tokens_divided=[]
        for word_to_tokenize in sentence_to_tokenize:
            temp=[]
            temp=self.tokenize_word(word_to_tokenize)
            res_tokens_divided=res_tokens_divided+temp
        return res_tokens_divided
    def write_vocab_to_file(self):
        with open(self.path_for_vocab_txt_out,"w") as file:
            for tokens in self.vocabulary:
                file.write(tokens+"\n")
    def extract_from_json(self,path_to_json):
        with open(path_to_json,"r") as f:
            test_data=json.load(f)
        return test_data
    def tokenize(self):
        test_data=self.extract_from_json(self.sentences_id_in_json)
        data_to_be_written={}
        for entry in test_data:
            id_sen=entry["id"]
            sen=entry["sentence"]
            data_to_be_written[id_sen]=self.tokenize_sentence(sen[:-1])



        out_file=self.token_id_out_json
        with open(out_file,"w") as f:
            json.dump(data_to_be_written,f,indent=4)
            
    

initialise the following way:\
corpus file path\
path of the output vocabulary (generated from the corpus)\
input sentences json\
output tokens json\
iterations\
max_vocab_size

In [9]:
test_class=WordPieceTokenizer("corpus.txt","vocabulary_out.txt","sample_test.json","sentences_to_token_file.json",100,90)
test_class.construct_vocabulary()
test_class.tokenize()


{'i': 341, 'stand': 1, 'here': 7, 'feel': 145, 'empty': 4, 'a': 74, 'class': 1, 'post': 3, 'count': 1, 'link': 1, 'href': 4, 'http': 5, 'mooshilu': 1, 'literally': 1, 'just': 28, 'text': 1, 'tychelle': 1, 'to': 97, 'see': 6, 'if': 9, 'she': 3, 'wants': 3, 'hang': 1, 'out': 6, 'because': 18, 'reading': 1, 'what': 11, 'wrote': 2, 'about': 22, 'my': 58, 'nonexistent': 1, 'social': 2, 'life': 15, 'made': 3, 'me': 31, 'so': 38, 'pathetic': 3, 'really': 14, 'regretful': 3, 'when': 17, 'hearing': 1, 'that': 58, 'shinae': 2, 'got': 3, 'married': 1, 'another': 1, 'man': 1, 'oh': 2, 'it': 52, 's': 2, 'sad': 5, 'hope': 3, 'alex': 1, 'and': 137, 'can': 10, 'be': 13, 'couple': 2, 'in': 46, 'real': 3, 'they': 12, 're': 1, 'perfect': 1, 'for': 35, 'each': 1, 'other': 3, 'believed': 1, 'was': 34, 'true': 2, 'love': 4, 'devastated': 5, 'wanted': 1, 'settle': 1, 'down': 3, 'have': 21, 'the': 92, 'whole': 2, 'marriage': 1, 'kids': 3, 'thing': 2, 'with': 24, 'him': 7, 'unimportant': 3, 'inadequate': 8, 'v